# ROI Classification Verification

This notebook verifies the ROI classification logic using synthetic data.

**Test Cases:**
1.  **Internal:** ROI inside cell, distance > 30px.
2.  **Outside:** ROI outside cell, distance > 30px (-31px).
3.  **Edge (Inside):** ROI inside cell, distance <= 30px.
4.  **Edge (Outside):** ROI outside cell, distance <= 30px.

In [ ]:
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt

# Mock ROI Object
class MockRoi:
    def __init__(self, y, x):
        self.centroid = (y, x)

# 1. Create Synthetic Mask (Circle)
size = 200
mask = np.zeros((size, size), dtype=np.uint8)
center = (100, 100)
radius = 50

y, x = np.ogrid[:size, :size]
mask_area = (x - center[1])**2 + (y - center[0])**2 <= radius**2
mask[mask_area] = 1

plt.imshow(mask, cmap='gray')
plt.title("Synthetic Cell Mask")
plt.show()

In [ ]:
# 2. Re-implement Logic functions for verification

def compute_signed_distance_map(mask):
    binary_mask = mask > 0
    dist_in = ndimage.distance_transform_edt(binary_mask)
    dist_out = ndimage.distance_transform_edt(~binary_mask)
    signed_dist = dist_in - dist_out
    return signed_dist

def classify_roi(dist_map, roi, edge_thr=30):
    y, x = roi.centroid
    d = dist_map[int(y), int(x)]
    
    if d > edge_thr:
        return "Internal", d
    elif d < -edge_thr:
        return "Outside", d
    else:
        return "Edge", d

In [ ]:
# 3. Run Tests
dist_map = compute_signed_distance_map(mask)

# Visual Check of Distance Map
plt.imshow(dist_map, cmap='RdBu')
plt.colorbar(label='Distance (px)')
plt.title("Signed Distance Map")
plt.show()

# Define Test Points
# Center is at (100, 100). Radius is 50.
# Boundary is at distance 0 (approx radius 50 from center).

test_cases = [
    # (y, x, expected_class, description)
    (100, 100, "Internal", "Center (Dist ~50 > 30)"),
    (100, 100+40, "Edge", "Inside near edge (Dist ~10 <= 30)"),
    (100, 100+55, "Edge", "Outside near edge (Dist ~-5 >= -30)"),
    (100, 100+90, "Outside", "Far Outside (Dist ~-40 < -30)"),
    (100, 100+25, "Edge", "Inside boundary case 25px (Dist ~25 <= 30)"), 
    (100, 100+19, "Internal", "Inside boundary case 31px (Dist ~31 > 30)") # 50-19=31
]

print("Running Tests...\n")
for y, x, expected, desc in test_cases:
    roi = MockRoi(y, x)
    cls, dist = classify_roi(dist_map, roi, edge_thr=30)
    
    status = "PASS" if cls == expected else "FAIL"
    print(f"{status} | {desc}: Got '{cls}' (Dist: {dist:.1f})")
    assert cls == expected, f"Expected {expected}, got {cls} for dist {dist}"